## Submission to the candiate test for the Vodafone Data Scientist position
 - Author: István Juhász
 - Position applied for:Data scientist
 - date:2019-04-13
 - Written using the anaconda package on a jupyter notebook. Executed on openSUSE Tumbelweed

In [1]:
import os
!cat /etc/os-release|grep -E 'PRETTY_NAME|VERSION_ID'
!python -V

VERSION_ID="20190512"
PRETTY_NAME="openSUSE Tumbleweed"
Python 3.7.3


## Preface and tools used.
While the problem could hardly be categorised as particularly difficult or of a niche subject in itself the problem nevertheless did present an additional hurdle for me to overcome. This being, ofcourse, is that I had been using R for data science and my Python work had been so far almost exclusively limited to solving automation problems. Thankfully, this language and tools have an immense and freely available literature.

The above notwithstanding, trying to complete this task to the best of my abilities had been a rather quite interesting and may I says 'fun' challenge and while the alloted time had been certainly a limiting factor, my only regret is that I haven't started using this language earlier.  

After a brief elaboration, of the packages recommended by <a href="Python Machine Learning Libraries That You Should Master">Navin Bondade</a>: Scikit-Learn classic,TensorFlow (neural networks), Keras (deep learning), LightGBM. I have opted to use the package explicitly permitted by the test instruction which also seemed to be the most approachable ( perhaps only to Tensorflow) and came with the largest user- and available knowledgebase. 

### Churn Prediction

Considering each customer as an individual revenue stream, they could be understood as an individual asset with their revenue streams along with acquisition and retention costs. An effective algorithm for identifying the clients in danger of churning could mitigate both the size of unnecessary retention and avoidable acquisition costs. 

I have relied on a handful of articles to solve this problem:
- <a href="https://towardsdatascience.com/machine-learning-powered-churn-analysis-for-modern-day-business-leaders-ad2177e1cb0d?gi=145fe87dc040">Machine Learning Powered Churn Analysis for Modern Day Business Leaders - Anupam Kundu</a>
- <a href="https://www.datascience.com/blog/churn-prediction-python">Introduction to Churn Prediction in Python - Seyed Sajjadi </a>
- <a href="https://towardsdatascience.com/hands-on-predict-customer-churn-5c2a42806266?gi=83aaf592063d">Hands-on: Predict Customer Churn</a>
- <a href="https://www.kaggle.com/pavanraj159/telecom-customer-churn-prediction">Telecom Customer Churn Prediction</a>

### Evaluation 

Among the many different measures of a model's skill, there are 4 base ratios(Sensitivity(true positive rate, Specificity (true negative rate), Precision and Recall(sensitivity) deserves higher importance: ROC  and Precision Recall. The former shows the rate of correctly identified true positives with the rate incorrectly identified true negative while the latter shows the percentage of true positives in the  selected items and the  percentage of captured positive items at any given threshold. The difference is, ofcourse, that the Precision and recall relies only on the positive cases and as such it is meaningful even in cases of strong class imbalance. 

### Exploratory Data Analysis

In [2]:
%%capture 
#preventing the installation data spoiling the document
import sys
!conda install  --yes --prefix {sys.prefix} -c conda-forge imbalanced-learn #sys.prefix prevents installation to defaulting to the default python version.
!conda install  --yes --prefix {sys.prefix} plotly
!conda install  --yes --prefix {sys.prefix} -c conda-forge  xgboost
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv(filepath_or_buffer=r"/home/letienne/Documents/Code/Python/VodaFoneDataScientistTest/telco.csv",sep=',',encoding='UTF-8',decimal='.')
df.head()

,region,tenure,age,marital,address,income,ed,employ,retire,gender,...,voice,pager,internet,callid,callwait,forward,confer,ebill,custcat,churn
0,2,13,44,1,9,64.0,4,5,0.0,0,...,0,0,0,0,0,1,0,0,1,1
1,3,11,33,1,7,136.0,5,5,0.0,0,...,1,1,0,1,1,1,1,0,4,1
2,3,68,52,1,24,116.0,1,29,0.0,1,...,0,0,0,1,1,0,1,0,3,0
3,2,33,33,0,12,33.0,2,0,0.0,1,...,0,0,0,0,0,0,0,0,1,1
4,2,23,30,1,9,30.0,1,2,0.0,0,...,0,0,0,1,0,1,1,0,3,0


In [4]:
#Piecharts are one of the most popular ways to represent simple distribution 
sizes = df['churn'].value_counts(sort = True)
colors = ["red","pink"] 
labels=["Retained","Churned"]
import matplotlib.pyplot as plt 
# Plot
plt.pie(sizes, explode=None, colors=colors,
        autopct='%1.1f%%', labels=labels,shadow=True, startangle=90)

plt.title('The size of the churned customers as a ratio of all observations.')
plt.show()

<Figure size 640x480 with 1 Axes>

## Data PreProcessing

In [5]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df = pd.read_csv(filepath_or_buffer=r"/home/letienne/Documents/Code/Python/VodaFoneDataScientistTest/telco.csv",sep=',',encoding='UTF-8',decimal='.')

col_Flags=['tollfree', 'equip', 'callcard', 'wireless','multline','voice','pager','internet','callid','callwait','forward','confer','ebill','churn']
col_Factors=['region', 'marital', 'retire', 'gender','custcat']
col_Ordinal=['ed']
col_Target = ['churn']
col_Continuous= [x for x in df.columns if x not in col_Factors + col_Ordinal + col_Flags]
for col in col_Factors:
    df[col] = df[col].astype('category')
for col in col_Flags:    
    df[col] = df[col].astype('uint8')
for col in col_Ordinal:    #LabelEncoding doesn't really have an effect on booleans coded in numerical variables but it is one of the ways to handle ordinal data without forgoing the additional information compared to factors.
    df[col] = le.fit_transform(df[col])
for col in col_Continuous:    
    df[col] = df[col].astype('float64')
df = pd.get_dummies(data = df,columns = col_Factors+col_Ordinal)

#Scaling Numerical columns. most models - with the notable exception of tree based algorithms - are highly sensitive to numerical data that had not been scaled and centered.   
std = StandardScaler()
scaled = std.fit_transform(df[col_Continuous])
scaled = pd.DataFrame(scaled,columns=col_Continuous)

df_original = df.copy()
df = df.drop(columns = col_Continuous,axis = 1)
df = df.merge(scaled,left_index=True,right_index=True,how = "left") 


## Variable summary
Thankfully, no missing values were found in this dataset.

In [6]:
import plotly.graph_objs as go
import plotly.offline as py #visualization
import warnings as warnings
py.init_notebook_mode(connected=True)#visualization
warnings.filterwarnings("ignore")

Nullvalues=pd.DataFrame(df_original.isnull().sum())
Nullvalues.rename(columns={0: "Missing"},inplace=True)

summary = (df_original[[i for i in df_original.columns]].
           describe().transpose().reset_index())

summary = summary.rename(columns = {"index" : "feature"})
summary = np.around(summary,3)
summary = pd.merge(summary, Nullvalues, left_on='feature',right_index=True)

val_lst = [summary['feature'], summary['count'],
           summary['mean'],summary['std'],
           summary['min'], summary['25%'],
           summary['50%'], summary['75%'], summary['max'],summary['Missing']]

#using branding compatible colours 
trace  = go.Table(header = dict(values = summary.columns.tolist(),
                                line = dict(color = ['#ffffff']),
                                fill = dict(color = ['#E60000']),
                                font = dict(color = ['#ffffff'])                                
                               ),
                  cells  = dict(values = val_lst,
                                line = dict(color = ['#506784']),
                                fill = dict(color = ["#ffffff",'#ffffff'])
                               ),
                  columnwidth = [200,60,100,100,60,60,80,80,80])
layout = go.Layout(dict(title = "Feature  Summary"))
figure = go.Figure(data=[trace],layout=layout)
py.iplot(figure)

# Model Building

### Synthetic Minority OverSampling technique
It seems that the target variable is a somewhat imbalanced. In such cases, increasing the share number of the minority outcome could improve certain models. One frequently used way to do this is oversampling, specifically the synthetic Minority Oversampling technique from the imbalanced-learning package.

In [429]:
from sklearn.model_selection import train_test_split
col_features    = [i for i in df.columns if i not in col_Target]
from imblearn.over_sampling import SMOTE

train_X,test_X,train_Y,test_Y = train_test_split(df[col_features],df[col_Target],
                                                                         test_size = .20 ,
                                                                         random_state = 1986)
os = SMOTE(random_state = 0)
resampled_X,resampled_Y = os.fit_sample(train_X,train_Y)
resampled_X = pd.DataFrame(data = os_smote_X,columns=col_features)
resampled_Y = pd.DataFrame(data = resampled_Y,columns=col_Target)

### Functions

In [430]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report, roc_auc_score, \
roc_curve,scorer,precision_score,recall_score 
from sklearn.model_selection import cross_val_score,RandomizedSearchCV

def GetMetricRecord(Model,ModelName,train_X,train_Y,test_x,test_y):
    Model.fit(train_X,train_Y)
    Y_hat   = Model.predict(test_x)
    #Probabilities = Model.predict_proba(Features)
    auroc = roc_auc_score(test_y.values.ravel(),Y_hat)
    acc=accuracy_score(test_y.values.ravel(),Y_hat)
    #auroc = cross_val_score(Model, X, y, cv=10, scoring='roc_auc').mean() # using crossvalidation to mitigate effects from the sampling
    recall  = recall_score(test_y.values.ravel(),Y_hat)
    return pd.DataFrame({'Accuracy':[acc],'AUC':[auroc],'RecallScore':[recall]},index=[ModelName])

### Simple Logistic Regression

In [431]:
from sklearn.linear_model import LogisticRegression
logit = LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

### Extreme Gradient Boosting
It recently became one of the most popular technique in machine learning, an ubiquitous feature of kaggle competitions. It has numerous advantages over standard boosting techniques like  built in paralelisation and cross-validation capability, the latter of which helps limiting exposure to variance in sampling and the former grants it an immense boost on servers and other high-core count workstations. 
In the below base model, of the two available booster models - tree and linear - , the tree one is used as it is known to be greatly outperforming the latter.

In [432]:
from xgboost import XGBClassifier

XGBModel=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
                    colsample_bytree=1, gamma=0, learning_rate=0.9, max_delta_step=0,
                    max_depth = 7, min_child_weight=1, missing=None, n_estimators=100,
                    n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
                    reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
                    silent=True, subsample=1)

### RandomForests

In [433]:
from sklearn.ensemble import RandomForestClassifier
RForest = RandomForestClassifier()

### Support Vector Machines

In [434]:
from sklearn.svm import SVC
SVCLinear  = SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
               decision_function_shape='ovr', degree=3, gamma=1.0, kernel='linear',
               max_iter=-1, probability=True, random_state=None, shrinking=True,
               tol=0.001, verbose=False)
SVCrbf  = SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0, #the default
               decision_function_shape='ovr', degree=3, gamma=1.0, kernel='rbf',  
               max_iter=-1, probability=True, random_state=None, shrinking=True,
               tol=0.001, verbose=False)

### Model Selection

In [435]:
ModelComparison = pd.DataFrame(columns=['ModelName', 'Accuracy', 'AUC','RecallScore'])
ModelComparison.set_index('ModelName',inplace=True)

ModelComparison=pd.concat([ModelComparison \
                            ,GetMetricRecord(SVCLinear,'SVCLinear',resampled_X,resampled_Y,test_X,test_Y) \
                            ,GetMetricRecord(SVCrbf,'SVCrbf',resampled_X,resampled_Y,test_X,test_Y) \
                            ,GetMetricRecord(XGBModel,'XGB1',resampled_X,resampled_Y,test_X,test_Y) \
                            ,GetMetricRecord(logit,'logit',resampled_X,resampled_Y,test_X,test_Y) \
                            ,GetMetricRecord(RForest,'RForest',resampled_X,resampled_Y,test_X,test_Y) \
                            #,GetMetricRecord(TunedRF,'TunedRF',resampled_X,resampled_Y,test_X,test_Y) \
                           
                          ],sort=False)

ModelTable = ModelComparison.copy() #values, not pointers
ModelTable.index.names = ['ModelName']
ModelTable.reset_index(level=0, inplace=True)

#ModelTable = ModelTable.rename(columns = {"index" : "ModelName"})
ModelTable = np.around(ModelTable,3)

val_lst = [ModelTable['ModelName']
           ,ModelTable['Accuracy']
           ,ModelTable['AUC']
           ,ModelTable['RecallScore']
          ]

#using branding compatible colours 
trace  = go.Table(header = dict(values = ModelTable.columns.tolist(),
                                line = dict(color = ['#ffffff']),
                                fill = dict(color = ['#E60000']),
                                font = dict(color = ['#ffffff'])                                
                               ),
                  cells  = dict(values = val_lst,
                                line = dict(color = ['#506784']),
                                fill = dict(color = ["#ffffff",'#ffffff'])
                               ),
                  columnwidth = [60,100,100,100])
layout = go.Layout(dict(title = "Model Measurements"))
figure = go.Figure(data=[trace],layout=layout)
py.iplot(figure)

### Second round:  Tuning the RandomForestClassifier

The above table shows promising results that warrant for parameter tuning - where applicable -  for the randomforest, logistic regression and linear support vector machines. Unfortunately time permitted only the RandomForestClassifier to be completed.

In [436]:
# number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]

# number of features at every split
max_features = ['auto', 'sqrt']

# max depth
max_depth = [int(x) for x in np.linspace(100, 500, num = 11)]
max_depth.append(None)

# create random grid
random_grid = {
 'n_estimators': n_estimators,
 'max_features': max_features,
 'max_depth': max_depth
 }
random_grid
# Random search of parameters
rfc_random = RandomizedSearchCV(estimator = RForest, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)

# Fit the model
rfc_random.fit(resampled_X,resampled_Y)
rfc_random.best_params_
# Tuned parameters
print(rfc_random.best_params_)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   37.7s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  4.7min finished


{'n_estimators': 200, 'max_features': 'auto', 'max_depth': 220}


In [437]:
# with the tuned models added

TunedRF = RandomForestClassifier(n_estimators=200, max_depth=220, max_features='auto')
ModelComparison = pd.DataFrame(columns=['ModelName', 'Accuracy', 'AUC','RecallScore'])
ModelComparison.set_index('ModelName',inplace=True)
#the various validation measures should have used some sort of crossvalidation
ModelComparison=pd.concat([ModelComparison \
                            ,GetMetricRecord(SVCLinear,'SVCLinear',resampled_X,resampled_Y,test_X,test_Y) \
                            ,GetMetricRecord(SVCrbf,'SVCrbf',resampled_X,resampled_Y,test_X,test_Y) \
                            ,GetMetricRecord(XGBModel,'XGB1',resampled_X,resampled_Y,test_X,test_Y) \
                            ,GetMetricRecord(logit,'logit',resampled_X,resampled_Y,test_X,test_Y) \
                            ,GetMetricRecord(RForest,'RForest',resampled_X,resampled_Y,test_X,test_Y) \
                            ,GetMetricRecord(TunedRF,'TunedRF',resampled_X,resampled_Y,test_X,test_Y) \
                           
                          ],sort=False)

ModelTable = ModelComparison.copy() #values, not pointers
ModelTable.index.names = ['ModelName']
ModelTable.reset_index(level=0, inplace=True)


ModelTable = np.around(ModelTable,3)

val_lst = [ModelTable['ModelName']
           ,ModelTable['Accuracy']
           ,ModelTable['AUC']
           ,ModelTable['RecallScore']
          ]

#using branding compatible colours 
trace  = go.Table(header = dict(values = ModelTable.columns.tolist(),
                                line = dict(color = ['#ffffff']),
                                fill = dict(color = ['#E60000']),
                                font = dict(color = ['#ffffff'])                                
                               ),
                  cells  = dict(values = val_lst,
                                line = dict(color = ['#506784']),
                                fill = dict(color = ["#ffffff",'#ffffff'])
                               ),
                  columnwidth = [60,100,100,100])
layout = go.Layout(dict(title = "Model Measurements"))
figure = go.Figure(data=[trace],layout=layout)
py.iplot(figure)

### Conclusion

The support vector machines are often regarded as one of the black box starting points that - albeit with being rather difficult to interpret - could be used as targets from where invaluable feature importance information could be extracted.

In this scenario, however,  even through the non-linear version had produced the highest accuracy scores, the other two measurements show that there is something seriously wrong with our model and thus at least requires a second look at.

The tuned Randomforest model had achieved better scores in most measures than its original base model further underlying the importance of tuning the other contenders and making it one of our best performing models.  

Surprisingly the performing model was the most basic logistic regression whose results are the only one which come close to being acceptable.

Thus, if an immediate decision were to be made, it would be logistic Regression that I would suggest going forward with but my recommendation would be continuing with the steps listed below:

### Recommended subsequent steps
There are several other potential issues that need to be verified before the algorithm could be put into production :

 - Not all models have (good) automatic feature selection.No numerical variable correlations were examined,the sparse variables were not removed. Removing variables with highly correlated and near-zero variance features should improve the variance of most models.
 - Parameter tuning, especially for the Extreme Gradient Boosting model.   
 - No neural network or KNN models had been tried.
 - The comparison of the feature importances of these models, especially the one with problematic results like the extreme gradient boost mode.  
 - The continuous variables could be binned, even if it is generally not recommended.
 - Preferably better visualisaton, graphs like ROC curves. 
